In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from IPython.display import display, clear_output
import pandas as pd
import time
import json

from itertools import product
from collections import namedtuple
from collections import OrderedDict

In [ ]:
# TODO: Add regularization, num_workers, and CUDA concepts


PyTorch allows us to seamlessly move data to and from our GPU as we preform computations inside our programs.

When we go to the GPU, we can use the cuda() method, and when we go to the CPU, we can use the cpu() method.

We can also use the to() method. To go to the GPU, we write to('cuda') and to go to the CPU, we write to('cpu'). The to() method is the preferred way mainly because it is more flexible. We'll see one example using using the first two, and then we'll default to always using the to() variant.

CPU 	GPU
cpu() 	cuda()
to('cpu') 	to('cuda')

To make use of our GPU during the training process, there are two essential requirements. These requirements are as follows, the data must be moved to the GPU, and the network must be moved to the GPU.

    Data on the GPU
    Network on the GPU

Data standardization is a specific type of normalization technique. It is sometimes referred to as z-score normalization. The z-score, a.k.a. standard score, is the transformed value for each data point.

To normalize a dataset using standardization, we take every value
inside the dataset and transform it to its correspondingvalue using the following formula:

z = (x - mean)/std

After performing this computation on every value inside our dataset, we have a new normalized dataset of values. The mean and standard deviation values are with respect to the dataset as a whole. 

<i>It's important to note that when we normalize a dataset, we typically group these operations by feature. This means that the mean and standard deviation values are relative to each feature set that's being normalized. If we are working with images, the features are the RGB color channels, so we normalize each color channel with respect to the mean and standard deviation values calculated across all pixels in every images for the respective color channel. In our case we only needs to
normalize a single color channel</i>

In [2]:
# Normalization typically occurs at the extraction and transform stages of the ETL process, we can pass the mean and std
# via the Normalize method as such:
# torchvision.transforms.Normalize(
#       [meanOfChannel1, meanOfChannel2, meanOfChannel3] 
#     , [stdOfChannel1, stdOfChannel2, stdOfChannel3] 
# )
# However, we dont have the mean and std of the channel we are working with and will need to calculate it

train_set = torchvision.datasets.FashionMNIST(
    root='/home/slabban/machine_learning_courses/datasets'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ]) 
)

In [7]:
# If we are dealing with a dataset with a total size that our computer can handle in one run we can simply do this:

loader = DataLoader(train_set, batch_size=len(train_set), num_workers=1)
images, labels = next(iter(loader))
images.mean(), images.std()

(tensor(0.2860), tensor(0.3530))

In [11]:
# Often times we will be dealing with huge datasets, we to tackle that case by spliting the set into batches
# and implementing the mean and std formulas

loader = DataLoader(train_set, batch_size=1000, num_workers=1)

num_of_pixels = len(train_set) * 28 * 28
total_sum = 0
for images, labels in loader: total_sum += images.sum()
mean = total_sum / num_of_pixels

sum_of_squared_error = 0
for images, labels in loader: 
    sum_of_squared_error += ((images - mean).pow(2)).sum()
std = torch.sqrt(sum_of_squared_error / num_of_pixels)

mean, std

(tensor(0.2860), tensor(0.3530))

In [22]:
# Lets now inlcude normalization in our extract and transform steps:

train_set = torchvision.datasets.FashionMNIST(
    root='/home/slabban/machine_learning_courses/datasets'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)])
)

In [24]:
# Th enew output of this mean and std is 0 and 1 respectively 

loader = DataLoader(
      train_set
    , batch_size=len(train_set)
    , num_workers=1
)
data = next(iter(loader))
data[0].mean(), data[0].std()

(tensor(-9.3774e-08), tensor(1.))